In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# --- STEP 0: Setup Environment & Dataset Path ---
import os
import numpy as np
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
import glob
import shutil
import tempfile
from pathlib import Path

# Kaggle environment already has required libraries pre-installed, no need for pip install

# Set dataset path for Kaggle environment
RAW_DATA_PATH = '/kaggle/input/breakhis/BreaKHis_v1'

# Set random seeds for reproducibility
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

print("Environment setup and libraries loaded.")
print(f"Raw data will be loaded from: {RAW_DATA_PATH}")


2025-09-22 06:35:01.619489: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758522901.958252      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758522902.049337      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Environment setup and libraries loaded.
Raw data will be loaded from: /kaggle/input/breakhis/BreaKHis_v1


In [2]:
# --- NEW CELL: Reorganize Nested Dataset Structure ---
def reorganize_breakhis_dataset(input_path, output_path=None):
    """
    Reorganize the nested BreakHis dataset into a standard structure
    that can be used by ImageDataGenerator.
    
    Parameters:
    input_path: Path to the BreakHis dataset root
    output_path: Where to create the organized dataset 
                 (default: /kaggle/working/BreaKHis_organized for persistence)
    
    Returns:
    Path to the organized dataset
    """
    if output_path is None:
        # Use Kaggle's persistent working directory
        output_path = "/kaggle/working/BreaKHis_organized"
        print(f"Using persistent directory: {output_path}")
    else:
        print(f"Using custom directory: {output_path}")
    
    os.makedirs(output_path, exist_ok=True)
    
    # Define the magnifications we want to use
    magnifications = ['40X', '100X', '200X', '400X']
    
    # Create directory structure: output_path/class/magnification/
    for cls in ['benign', 'malignant']:
        for mag in magnifications:
            os.makedirs(os.path.join(output_path, cls, mag), exist_ok=True)
    
    # Find all image files and organize them
    image_extensions = ['*.png', '*.jpg', '*.jpeg', '*.tif', '*.tiff']
    image_count = 0
    
    for extension in image_extensions:
        # Search for all images in the nested structure
        search_pattern = os.path.join(input_path, '**', extension)
        image_paths = glob.glob(search_pattern, recursive=True)
        
        for image_path in image_paths:
            try:
                # Extract class and magnification from path
                path_parts = Path(image_path).parts
                
                # Determine class (benign or malignant)
                if 'benign' in path_parts:
                    cls = 'benign'
                elif 'malignant' in path_parts:
                    cls = 'malignant'
                else:
                    continue  # Skip if class not found
                
                # Determine magnification
                mag = None
                for m in magnifications:
                    if m in path_parts:
                        mag = m
                        break
                
                if mag is None:
                    continue  # Skip if magnification not found
                
                # Copy image to organized structure
                filename = os.path.basename(image_path)
                dest_path = os.path.join(output_path, cls, mag, filename)
                
                # Avoid overwriting existing files
                if not os.path.exists(dest_path):
                    shutil.copy2(image_path, dest_path)
                    image_count += 1
                    
            except Exception as e:
                print(f"Error processing {image_path}: {str(e)}")
                continue
    
    print(f"✅ Organized {image_count} images into {output_path}")
    return output_path


# Reorganize the dataset into Kaggle's working directory
DATA_PATH = reorganize_breakhis_dataset(RAW_DATA_PATH)
print(f"Organized dataset available at: {DATA_PATH}")


Using persistent directory: /kaggle/working/BreaKHis_organized
✅ Organized 7909 images into /kaggle/working/BreaKHis_organized
Organized dataset available at: /kaggle/working/BreaKHis_organized


In [3]:
# --- NEW CELL: Verify Dataset Organization ---
def verify_dataset_structure(data_path):
    """Verify that the dataset is properly organized."""
    import matplotlib.pyplot as plt
    from matplotlib.image import imread
    
    classes = ['benign', 'malignant']
    magnifications = ['40X', '100X', '200X', '400X']
    
    print("Dataset structure verification:")
    print("=" * 50)
    
    for cls in classes:
        for mag in magnifications:
            class_mag_path = os.path.join(data_path, cls, mag)
            if os.path.exists(class_mag_path):
                image_files = [f for f in os.listdir(class_mag_path) 
                              if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff'))]
                print(f"{cls}/{mag}: {len(image_files)} images")
            else:
                print(f"{cls}/{mag}: Directory does not exist")

# Verify the organized dataset
verify_dataset_structure(DATA_PATH)

Dataset structure verification:
benign/40X: 625 images
benign/100X: 644 images
benign/200X: 623 images
benign/400X: 588 images
malignant/40X: 1370 images
malignant/100X: 1437 images
malignant/200X: 1390 images
malignant/400X: 1232 images


In [4]:
# --- STEP 1: Data Preparation ---
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import preprocess_input

# Choose which magnification to use
SELECTED_MAGNIFICATION = '400X'  # You can change this to '40X', '100X', '200X', or '400X'

# Use the organized dataset path with selected magnification
PROCESSED_DATA_PATH = os.path.join(DATA_PATH, SELECTED_MAGNIFICATION)

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
IMG_SHAPE = IMAGE_SIZE + (3,)

# Training data generator with augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Validation data generator without augmentation
validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

try:
    train_generator = train_datagen.flow_from_directory(
        DATA_PATH,  # Use the root organized path to include all magnifications
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='training',
        seed=SEED,
        shuffle=True
    )

    validation_generator = validation_datagen.flow_from_directory(
        DATA_PATH,  # Use the root organized path to include all magnifications
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='validation',
        seed=SEED,
        shuffle=False
    )
    
    # Check class distribution
    print(f"Training samples: {train_generator.samples}")
    print(f"Validation samples: {validation_generator.samples}")
    print(f"Classes: {train_generator.class_indices}")
    
except Exception as e:
    print(f"Error creating data generators: {str(e)}")
    # Fallback to the original dataset path
    print("Falling back to original dataset structure...")
    
    # Try with the original nested structure
    train_generator = train_datagen.flow_from_directory(
        RAW_DATA_PATH,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='training',
        seed=SEED,
        shuffle=True
    )

    validation_generator = validation_datagen.flow_from_directory(
        RAW_DATA_PATH,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='validation',
        seed=SEED,
        shuffle=False
    )

Found 6328 images belonging to 2 classes.
Found 1581 images belonging to 2 classes.
Training samples: 6328
Validation samples: 1581
Classes: {'benign': 0, 'malignant': 1}


In [5]:
# --- Calculate Class Weights ---
try:
    # Calculate class weights to handle imbalance
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(train_generator.classes),
        y=train_generator.classes
    )
    class_weights_dict = dict(enumerate(class_weights))
    print(f"Class weights: {class_weights_dict}")
    
except Exception as e:
    print(f"Error calculating class weights: {str(e)}")
    class_weights_dict = {0: 1.0, 1: 1.0}  # Default weights if calculation fails

Class weights: {0: 1.594758064516129, 1: 0.7283609576427256}


In [6]:
# --- STEP 2: Build the ResNet50 Model ---
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

print("Building ResNet50 model...")

try:
    # Load pre-trained ResNet50 model without the top classification layer
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
    base_model.trainable = False  # Freeze the base model

    # Add a new top layer for our classification task with improved architecture
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)  # Reduced dropout rate
    x = Dense(256, activation='relu')(x)  # Added intermediate layer
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    # Combine base model and new top layer
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model with improved learning rate
    model.compile(optimizer=Adam(learning_rate=0.0005),  # Reduced learning rate
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), 
                           tf.keras.metrics.Recall(name='recall')])

    model.summary()
    
except Exception as e:
    print(f"Error building model: {str(e)}")
    # Fallback to simpler model if ResNet50 fails
    from tensorflow.keras.applications import MobileNetV2
    
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
    base_model.trainable = False
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    print("Fallback model (MobileNetV2) created due to ResNet50 error")
    model.summary()

Building ResNet50 model...


I0000 00:00:1758523120.628076      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1758523120.628869      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,121,729 (92.02 MB)

 Trainable params: 529,409 (2.02 MB)

 Non-trainable params: 23,592,320 (90.00 MB)

In [ ]:
# --- STEP 3: Train the Model ---
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger

print("Training the model...")

try:
    # Define improved callbacks with better monitoring
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=8,  # Increased patience
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.2, 
        patience=5,  # Increased patience
        min_lr=0.00001,
        verbose=1
    )
    
    checkpoint = ModelCheckpoint(
        'best_model.keras', 
        monitor='val_accuracy', 
        save_best_only=True, 
        mode='max',
        verbose=1
    )
    
    csv_logger = CSVLogger('training_log.csv')

    # Calculate steps per epoch with validation
    steps_per_epoch = max(1, train_generator.samples // BATCH_SIZE)
    validation_steps = max(1, validation_generator.samples // BATCH_SIZE)

    print(f"Steps per epoch: {steps_per_epoch}")
    print(f"Validation steps: {validation_steps}")

    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=[early_stopping, reduce_lr, checkpoint, csv_logger],
        class_weight=class_weights_dict,
        verbose=1
    )
    
except Exception as e:
    print(f"Error during training: {str(e)}")
    # Try with smaller batch size if memory error
    if "memory" in str(e).lower() or "alloc" in str(e).lower():
        print("Memory error detected, trying with smaller batch size...")
        BATCH_SIZE = 16
        train_generator.batch_size = BATCH_SIZE
        validation_generator.batch_size = BATCH_SIZE
        
        steps_per_epoch = max(1, train_generator.samples // BATCH_SIZE)
        validation_steps = max(1, validation_generator.samples // BATCH_SIZE)
        
        history = model.fit(
            train_generator,
            steps_per_epoch=steps_per_epoch,
            epochs=20,
            validation_data=validation_generator,
            validation_steps=validation_steps,
            callbacks=[early_stopping, reduce_lr, checkpoint, csv_logger],
            class_weight=class_weights_dict,
            verbose=1
        )

### Model Training Results

- **Best Validation Accuracy:** 92.22%
- **Validation Loss at Best Accuracy:** 0.2004
- **Training Accuracy at Last Epoch:** 93.75%
- **Training Loss at Last Epoch:** 0.1427
- **Validation Precision / Recall:** 0.976 / 0.9086


In [ ]:
# --- STEP 4: Fine-Tuning ---
print("Fine-tuning the model...")

try:
    base_model.trainable = True

    # Fine-tune from the last few layers
    fine_tune_at = 140
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # Use even lower learning rate for fine-tuning
    model.compile(optimizer=Adam(learning_rate=0.00005),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), 
                           tf.keras.metrics.Recall(name='recall')])

    # Get the last epoch from initial training
    initial_epoch = len(history.history['loss'])

    history_fine_tune = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=initial_epoch + 10,
        initial_epoch=initial_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=[early_stopping, reduce_lr, checkpoint, csv_logger],
        class_weight=class_weights_dict,
        verbose=1
    )
    
except Exception as e:
    print(f"Error during fine-tuning: {str(e)}")
    print("Skipping fine-tuning and proceeding with evaluation...")
    history_fine_tune = history  # Use original history if fine-tuning fails

# 🩺 Breast Cancer Classification – Fine-Tuned ResNet50

**Model:** ResNet50 (pretrained + fine-tuned last layers)  
**Input Size:** 224 × 224 × 3  
**Training:** 30 epochs, with data augmentation & class weights  

## ✅ Performance Metrics (Best Epoch)

- **Validation Accuracy:** 95.54%  
- **Validation Loss:** 0.1360  
- **Precision:** 97.90%  
- **Recall:** 95.52%  

## 📌 Key Takeaways

- Fine-tuning improved validation accuracy from ~92% → **95.5%**  
- High precision & recall indicate strong performance on both classes  
- EarlyStopping & ModelCheckpoint ensured the **best model** is preserved


In [ ]:
# --- Plot Training History ---
import matplotlib.pyplot as plt

try:
    # Combine histories if fine-tuning was successful
    if 'history_fine_tune' in locals() and history_fine_tune != history:
        full_history = {
            'accuracy': history.history['accuracy'] + history_fine_tune.history['accuracy'],
            'val_accuracy': history.history['val_accuracy'] + history_fine_tune.history['val_accuracy'],
            'loss': history.history['loss'] + history_fine_tune.history['loss'],
            'val_loss': history.history['val_loss'] + history_fine_tune.history['val_loss']
        }
    else:
        full_history = history.history

    # Plot training history
    plt.figure(figsize=(12, 5), dpi=100)

    plt.subplot(1, 2, 1)
    plt.plot(full_history['accuracy'], label='Training Accuracy')
    plt.plot(full_history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='lower right')
    plt.grid(True, alpha=0.3)

    plt.subplot(1, 2, 2)
    plt.plot(full_history['loss'], label='Training Loss')
    plt.plot(full_history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('training_history.png', dpi=150, bbox_inches='tight')
    plt.show()
    
except Exception as e:
    print(f"Error plotting training history: {str(e)}")
    

In [ ]:
# --- STEP 5: Evaluation and Results Export ---
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, auc, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd

print("Evaluating the final model...")

try:
    # Get predictions and true labels with error handling
    validation_generator.reset()
    
    # Predict in batches to handle memory issues
    y_pred_proba = []
    y_true = []
    
    for i in range(validation_steps + 1):
        try:
            X_batch, y_batch = next(validation_generator)
            y_true.extend(y_batch)
            preds = model.predict(X_batch, verbose=0)
            y_pred_proba.extend(preds.flatten())
        except StopIteration:
            break
        except Exception as e:
            print(f"Error in batch {i}: {str(e)}")
            continue
    
    # Convert to arrays
    y_pred_proba = np.array(y_pred_proba)
    y_true = np.array(y_true)
    
    # Handle case where predictions and true labels might have different lengths
    min_length = min(len(y_pred_proba), len(y_true))
    y_pred_proba = y_pred_proba[:min_length]
    y_true = y_true[:min_length]
    
    y_pred = (y_pred_proba > 0.5).astype(int)

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    print("\nConfusion Matrix:")
    print(cm)

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=validation_generator.class_indices.keys()))

    # ROC AUC Score
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    print(f"\nROC AUC Score: {roc_auc:.4f}")

    # Calculate precision, recall, f1-score with error handling
    try:
        tn, fp, fn, tp = cm.ravel()
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    except ValueError:
        # Handle case where cm.ravel() doesn't return 4 values
        precision, recall, f1_score = 0, 0, 0
        
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1_score:.4f}")

    # Plot Confusion Matrix
    plt.figure(figsize=(8, 6), dpi=100)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=validation_generator.class_indices.keys(),
                yticklabels=validation_generator.class_indices.keys())
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig('confusion_matrix.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Plot ROC Curve
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    plt.figure(figsize=(8, 6), dpi=100)
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.grid(True, alpha=0.3)
    plt.savefig('roc_curve.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Save model and metrics
    model.save('breast_cancer_resnet50_final.keras')
    print("Model saved as 'breast_cancer_resnet50_final.keras'")

    metrics = {
        'accuracy': float(np.mean(y_pred == y_true)),
        'auc': float(roc_auc),
        'precision': float(precision),
        'recall': float(recall),
        'f1_score': float(f1_score),
        'confusion_matrix': cm.tolist()
    }

    with open('metrics.json', 'w') as f:
        json.dump(metrics, f, indent=4)

    # Save predictions for further analysis
    predictions_df = pd.DataFrame({
        'true_label': y_true,
        'predicted_prob': y_pred_proba,
        'predicted_label': y_pred
    })
    predictions_df.to_csv('predictions.csv', index=False)

    print("Training completed and results saved!")
    
except Exception as e:
    print(f"Error during evaluation: {str(e)}")
    # Try to save at least the model
    try:
        model.save('breast_cancer_resnet50_final.keras')
        print("Model saved despite evaluation errors")
    except:
        print("Could not save model")

In [ ]:
# --- STEP 2: Build the EfficientNetV2 Model ---
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

print("Building EfficientNetV2 model...")

try:
    # Load pre-trained EfficientNetV2 model without the top classification layer
    base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
    base_model.trainable = False  # Freeze the base model

    # Add a new top layer for classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    # Combine base model and new top layer
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0005),
                  loss='binary_crossentropy',
                  metrics=['accuracy',
                           tf.keras.metrics.Precision(name='precision'),
                           tf.keras.metrics.Recall(name='recall')])

    model.summary()

except Exception as e:
    print(f"Error building EfficientNetV2 model: {str(e)}")
    # Fallback to MobileNetV2 if EfficientNetV2 fails
    from tensorflow.keras.applications import MobileNetV2
    
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print("Fallback model (MobileNetV2) created due to EfficientNetV2 error")
    model.summary()


In [ ]:
# --- STEP 3: Train the EfficientNetV2 Model ---
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger

print("Training the EfficientNetV2 model...")

try:
    # Define improved callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss', 
        patience=8, 
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.2, 
        patience=5, 
        min_lr=1e-5,
        verbose=1
    )
    
    checkpoint = ModelCheckpoint(
        'best_efficientnetv2_model.keras', 
        monitor='val_accuracy', 
        save_best_only=True, 
        mode='max',
        verbose=1
    )
    
    csv_logger = CSVLogger('efficientnetv2_training_log.csv')

    # Steps per epoch
    steps_per_epoch = max(1, train_generator.samples // BATCH_SIZE)
    validation_steps = max(1, validation_generator.samples // BATCH_SIZE)

    print(f"Steps per epoch: {steps_per_epoch}")
    print(f"Validation steps: {validation_steps}")

    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=[early_stopping, reduce_lr, checkpoint, csv_logger],
        class_weight=class_weights_dict,
        verbose=1
    )

except Exception as e:
    print(f"Error during training: {str(e)}")
    # Fallback to smaller batch size if memory error
    if "memory" in str(e).lower() or "alloc" in str(e).lower():
        print("Memory error detected, retrying with smaller batch size...")
        BATCH_SIZE = 16
        train_generator.batch_size = BATCH_SIZE
        validation_generator.batch_size = BATCH_SIZE
        
        steps_per_epoch = max(1, train_generator.samples // BATCH_SIZE)
        validation_steps = max(1, validation_generator.samples // BATCH_SIZE)
        
        history = model.fit(
            train_generator,
            steps_per_epoch=steps_per_epoch,
            epochs=20,
            validation_data=validation_generator,
            validation_steps=validation_steps,
            callbacks=[early_stopping, reduce_lr, checkpoint, csv_logger],
            class_weight=class_weights_dict,
            verbose=1
        )


In [ ]:
# --- STEP 4: Fine-Tuning ---
print("Fine-tuning the model...")

try:
    # Unfreeze base model for fine-tuning
    base_model.trainable = True

    # Freeze only the first "fine_tune_at" layers
    fine_tune_at = 140
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # Recompile with a lower learning rate
    model.compile(
        optimizer=Adam(learning_rate=5e-5),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )

    # Resume training from where we left off
    initial_epoch = len(history.history['loss'])

    history_fine_tune = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=initial_epoch + 10,
        initial_epoch=initial_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=[early_stopping, reduce_lr, checkpoint, csv_logger],
        class_weight=class_weights_dict,
        verbose=1
    )

except Exception as e:
    print(f"\n⚠️ Error during fine-tuning: {str(e)}")
    print("➡️ Skipping fine-tuning and using previous training history instead.")
    history_fine_tune = history  # fallback to previous history


🩺 Breast Cancer Classification – Fine-Tuned EfficientNetV2
Model: EfficientNetV2B0 (pretrained + fine-tuned last layers)
Input Size: 224 × 224 × 3
Training: 30 epochs, with data augmentation & class weights
✅ Performance Metrics (Best Epoch)
Validation Accuracy: 95.09%
Validation Loss: 0.1444
Precision: 97.88%
Recall: 94.87%
📌 Key Takeaways
Fine-tuning improved validation accuracy from ~92% → 95.1%
High precision & recall indicate strong performance on both classes
EarlyStopping & ModelCheckpoint ensured the best model is pres

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import tensorflow as tf

# --- STEP 1: Combine Histories (initial + fine-tune) ---
def combine_histories(history, history_fine_tune):
    combined_history = {}
    for key in history.history.keys():
        combined_history[key] = history.history[key] + history_fine_tune.history.get(key, [])
    return combined_history

combined_history = combine_histories(history, history_fine_tune)

# --- STEP 2: Plot Training Curves ---
def plot_history(combined_history):
    metrics = ['accuracy', 'loss', 'precision', 'recall']
    plt.figure(figsize=(16, 12))
    
    for i, metric in enumerate(metrics):
        plt.subplot(2, 2, i+1)
        plt.plot(combined_history[metric], label=f"Train {metric}")
        plt.plot(combined_history[f"val_{metric}"], label=f"Val {metric}")
        plt.xlabel("Epochs")
        plt.ylabel(metric.capitalize())
        plt.title(f"Training vs Validation {metric.capitalize()}")
        plt.legend()
        plt.grid(True)

    plt.tight_layout()
    plt.show()

plot_history(combined_history)

# --- STEP 3: Evaluate Best Model on Validation/Test Set ---
print("\nLoading best saved model for evaluation...")
best_model = tf.keras.models.load_model("best_efficientnetv2_model.keras")

val_loss, val_acc, val_prec, val_rec = best_model.evaluate(validation_generator, verbose=1)
print(f"\nValidation Results -> Acc: {val_acc:.4f}, Precision: {val_prec:.4f}, Recall: {val_rec:.4f}")

# --- STEP 4: Classification Report + Confusion Matrix ---
print("\nGenerating predictions for confusion matrix...")
y_true = validation_generator.classes
y_pred_prob = best_model.predict(validation_generator, verbose=1)
y_pred = (y_pred_prob > 0.5).astype(int).ravel()

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=list(validation_generator.class_indices.keys())))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=validation_generator.class_indices.keys())
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
#  ---  Build the Vision Transformer (ViT) Model ---
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

print("Building Vision Transformer (ViT) model...")

try:
    from tensorflow.keras.applications import vit_b16  # ✅ Requires TF/Keras >= 2.15

    base_model = vit_b16.ViT_B16(
        include_top=False,
        pretrained="imagenet21k+imagenet2012",
        input_shape=IMG_SHAPE,
        pooling="avg"
    )
    base_model.trainable = False

    # Add custom head
    x = base_model.output
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile
    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss="binary_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(name="precision"), tf.keras.metrics.Recall(name="recall")]
    )

    model.summary()

except Exception as e:
    print(f"Error building ViT: {str(e)}")
    print("⚠️ Falling back to MobileNetV2...")

    from tensorflow.keras.applications import MobileNetV2
    base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=IMG_SHAPE)
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    model.summary()


In [ ]:
# ---  Train the Vision Transformer (ViT) Model ---
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger

print("Training the Vision Transformer (ViT) model...")

try:
    # Define improved callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=8,
        restore_best_weights=True,
        verbose=1
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-5,
        verbose=1
    )

    checkpoint = ModelCheckpoint(
        'best_vit_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )

    csv_logger = CSVLogger('vit_training_log.csv')

    # Steps per epoch
    steps_per_epoch = max(1, train_generator.samples // BATCH_SIZE)
    validation_steps = max(1, validation_generator.samples // BATCH_SIZE)

    print(f"Steps per epoch: {steps_per_epoch}")
    print(f"Validation steps: {validation_steps}")

    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=[early_stopping, reduce_lr, checkpoint, csv_logger],
        class_weight=class_weights_dict,
        verbose=1
    )

except Exception as e:
    print(f"Error during training: {str(e)}")
    # Fallback to smaller batch size if memory error
    if "memory" in str(e).lower() or "alloc" in str(e).lower():
        print("Memory error detected, retrying with smaller batch size...")
        BATCH_SIZE = 16
        train_generator.batch_size = BATCH_SIZE
        validation_generator.batch_size = BATCH_SIZE

        steps_per_epoch = max(1, train_generator.samples // BATCH_SIZE)
        validation_steps = max(1, validation_generator.samples // BATCH_SIZE)

        history = model.fit(
            train_generator,
            steps_per_epoch=steps_per_epoch,
            epochs=20,
            validation_data=validation_generator,
            validation_steps=validation_steps,
            callbacks=[early_stopping, reduce_lr, checkpoint, csv_logger],
            class_weight=class_weights_dict,
            verbose=1
        )


In [ ]:
# --- STEP 4: Fine-Tuning ---
print("Fine-tuning the ViT model...")

try:
    # Unfreeze the base model
    base_model.trainable = True

    # Define how many layers to keep frozen
    fine_tune_at = 140 if len(base_model.layers) > 140 else len(base_model.layers) // 2
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # Recompile with a smaller LR
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
        loss="binary_crossentropy",
        metrics=[
            "accuracy",
            tf.keras.metrics.Precision(name="precision"),
            tf.keras.metrics.Recall(name="recall")
        ]
    )

    # Resume training
    initial_epoch = len(history.history["loss"])

    history_fine_tune = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=initial_epoch + 10,   # add 10 more epochs
        initial_epoch=initial_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=[early_stopping, reduce_lr, checkpoint, csv_logger],
        class_weight=class_weights_dict,
        verbose=1
    )

except Exception as e:
    print(f"\n⚠️ Error during fine-tuning: {str(e)}")
    print("➡️ Skipping fine-tuning and using previous training history instead.")
    history_fine_tune = history  # fallback to previous training history


# 🩺 Breast Cancer Classification – Vision Transformer (ViT)

**Model:** Vision Transformer (ViT-B16)  
**Input Size:** 224 × 224 × 3  
**Training:** 20 epochs + fine-tuning 10 epochs, with data augmentation & class weights  

## ✅ Initial Training Performance

- **Best Validation Accuracy:** 83.61%  
- **Validation Loss:** 0.4253  

## ✅ Fine-Tuning Performance (Backbone Unfrozen)

- **Best Validation Accuracy:** 86.22%  
- **Validation Loss:** 0.4033  
- **Precision:** 86.64%  
- **Recall:** 94.40%  

## 📌 Key Takeaways

- Fine-tuning improved validation accuracy from **83.6% → 86.2%**  
- High recall indicates model effectively identifies malignant cases  
- EarlyStopping & ModelCheckpoint preserved the **best fine-tuned ViT model**


In [ ]:
from tensorflow.keras.models import load_model

best_vit_model = load_model("best_vit_model.keras")
val_loss, val_acc, val_prec, val_rec = best_vit_model.evaluate(validation_generator, verbose=1)
print(f"Validation -> Acc: {val_acc:.4f}, Precision: {val_prec:.4f}, Recall: {val_rec:.4f}")


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Predictions
val_preds = (best_vit_model.predict(validation_generator) > 0.5).astype("int32")
true_labels = validation_generator.classes

# Confusion matrix
cm = confusion_matrix(true_labels, val_preds)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=validation_generator.class_indices.keys(), yticklabels=validation_generator.class_indices.keys())
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("ViT Confusion Matrix")
plt.show()

# Classification report
print(classification_report(true_labels, val_preds, target_names=validation_generator.class_indices.keys()))


In [ ]:
plt.plot(history_fine_tune.history['accuracy'], label='Train Acc')
plt.plot(history_fine_tune.history['val_accuracy'], label='Val Acc')
plt.plot(history_fine_tune.history['loss'], label='Train Loss')
plt.plot(history_fine_tune.history['val_loss'], label='Val Loss')
plt.legend()
plt.title("ViT Fine-Tuning Performance")
plt.show()


In [ ]:
# --- STEP 1: Import Libraries ---
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger

print("Building DenseNet201 model...")

# --- STEP 2: Build the DenseNet201 Model ---
try:
    base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
    base_model.trainable = False  # Freeze base model

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss='binary_crossentropy',
        metrics=['accuracy',
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall')]
    )

    model.summary()

except Exception as e:
    print(f"Error building DenseNet201: {e}")
    # Fallback to smaller model (MobileNetV2)
    from tensorflow.keras.applications import MobileNetV2
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    print("Fallback model (MobileNetV2) created due to DenseNet201 error")
    model.summary()

# --- STEP 3: Setup Callbacks ---
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-5, verbose=1)
checkpoint = ModelCheckpoint('best_densenet201_model.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
csv_logger = CSVLogger('densenet201_training_log.csv')

# --- STEP 4: Train the Model ---
steps_per_epoch = max(1, train_generator.samples // BATCH_SIZE)
validation_steps = max(1, validation_generator.samples // BATCH_SIZE)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping, reduce_lr, checkpoint, csv_logger],
    class_weight=class_weights_dict,
    verbose=1
)

# --- STEP 5: Fine-tuning (Optional) ---
try:
    base_model.trainable = True
    fine_tune_at = 140
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    model.compile(
        optimizer=Adam(learning_rate=5e-5),
        loss='binary_crossentropy',
        metrics=['accuracy',
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall')]
    )

    initial_epoch = len(history.history['loss'])

    history_fine = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=initial_epoch + 10,
        initial_epoch=initial_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=[early_stopping, reduce_lr, checkpoint, csv_logger],
        class_weight=class_weights_dict,
        verbose=1
    )

except Exception as e:
    print(f"⚠️ Fine-tuning skipped: {e}")
    history_fine = history


# 🩺 Breast Cancer Classification – DenseNet201 Results

**Model:** DenseNet201 (pretrained on ImageNet, fine-tuned top layers)  
**Input Size:** 224 × 224 × 3  
**Epochs:** 21 (including fine-tuning)  
**Optimizer:** Adam  
**Loss Function:** Binary Crossentropy  

---

## ✅ Performance Metrics (Best Epoch)

- **Validation Accuracy:** 93.81%  
- **Validation Loss:** 0.1760  
- **Precision:** 95.01%  
- **Recall:** 95.99%  

---

## 📈 Training Observations

- Training and validation accuracy gradually improved after fine-tuning.  
- Early stopping prevented overfitting.  
- Precision and recall curves indicate **balanced performance** for both benign and malignant classes.  

---

## 📌 Key Takeaways

- DenseNet201 captures **rich features** due to dense connectivity.  
- Fine-tuning the deeper layers boosted **adaptation to the dataset**.  
- Compared to ResNet50, DenseNet201 shows **slightly higher recall** and comparable accuracy.  
- Further improvement is possible with **data augmentation** or **model ensembling**.


In [ ]:
# --- STEP 6: Visualizations (Fixed for generator mismatch) ---

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import seaborn as sns
import numpy as np

# 1️⃣ Plot training & validation accuracy/loss
plt.figure(figsize=(14,5))

# Accuracy
plt.subplot(1,2,1)
plt.plot(history_fine.history['accuracy'], label='Train Accuracy')
plt.plot(history_fine.history['val_accuracy'], label='Validation Accuracy')
plt.title('DenseNet201 Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss
plt.subplot(1,2,2)
plt.plot(history_fine.history['loss'], label='Train Loss')
plt.plot(history_fine.history['val_loss'], label='Validation Loss')
plt.title('DenseNet201 Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# 2️⃣ Confusion matrix & classification report
# Predict all validation data without using steps
val_generator = validation_generator
val_generator.reset()  # reset generator index
val_preds = model.predict(val_generator, verbose=1)
val_preds_classes = (val_preds > 0.5).astype(int).ravel()  # flatten to 1D

# Get true labels matching predictions
val_labels = val_generator.classes[:len(val_preds_classes)]

# Confusion matrix
cm = confusion_matrix(val_labels, val_preds_classes)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=val_generator.class_indices, 
            yticklabels=val_generator.class_indices)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('DenseNet201 Confusion Matrix')
plt.show()

# Classification report
print("Classification Report:\n")
print(classification_report(val_labels, val_preds_classes, target_names=val_generator.class_indices))

# 3️⃣ ROC Curve
fpr, tpr, thresholds = roc_curve(val_labels, val_preds)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, label=f'DenseNet201 ROC (AUC = {roc_auc:.2f})', color='darkorange')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# --- Precision, Recall, F1-score Bar Plot ---
from sklearn.metrics import precision_recall_fscore_support

metrics = precision_recall_fscore_support(val_labels, val_preds_classes)
classes = list(val_generator.class_indices.keys())

precision = metrics[0]
recall = metrics[1]
f1 = metrics[2]

x = np.arange(len(classes))
width = 0.2

plt.figure(figsize=(8,5))
plt.bar(x - width, precision, width, label='Precision', color='skyblue')
plt.bar(x, recall, width, label='Recall', color='lightgreen')
plt.bar(x + width, f1, width, label='F1-score', color='salmon')
plt.xticks(x, classes)
plt.ylim(0,1)
plt.ylabel('Score')
plt.title('DenseNet201 Class-wise Metrics')
plt.legend()
plt.show()


In [ ]:
# --- : Build Hybrid Model (EfficientNetV2 + ViT) ---
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, concatenate
from tensorflow.keras.optimizers import Adam

print("Building hybrid EfficientNetV2 + ViT model...")

try:
    # --- 1. Inputs ---
    inputs = Input(shape=IMG_SHAPE)

    # --- 2. EfficientNetV2 branch ---
    from tensorflow.keras.applications import EfficientNetV2B0
    enet_base = EfficientNetV2B0(weights='imagenet', include_top=False, input_tensor=inputs)
    enet_base.trainable = False
    enet_features = GlobalAveragePooling2D()(enet_base.output)
    enet_features = BatchNormalization()(enet_features)
    enet_features = Dropout(0.3)(enet_features)

    # --- 3. Vision Transformer branch ---
    try:
        from tensorflow.keras.applications import vit_b16  # Requires TF >= 2.15
        vit_base = vit_b16.ViT_B16(
            include_top=False,
            pretrained="imagenet21k+imagenet2012",
            input_shape=IMG_SHAPE,
            pooling="avg"
        )
        vit_base.trainable = False
        vit_features = vit_base(inputs)
        vit_features = BatchNormalization()(vit_features)
        vit_features = Dropout(0.3)(vit_features)

    except Exception as e:
        print(f"ViT import failed, using EfficientNetV2 as fallback for both branches: {str(e)}")
        vit_features = enet_features  # fallback to using EfficientNet features for ViT branch

    # --- 4. Concatenate features ---
    combined = concatenate([enet_features, vit_features])

    # --- 5. Dense layers for classification ---
    x = Dense(512, activation='relu')(combined)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    # --- 6. Create model ---
    hybrid_model = Model(inputs=inputs, outputs=outputs)

    # --- 7. Compile ---
    hybrid_model.compile(
        optimizer=Adam(learning_rate=0.0005),
        loss='binary_crossentropy',
        metrics=['accuracy', 
                 tf.keras.metrics.Precision(name='precision'), 
                 tf.keras.metrics.Recall(name='recall')]
    )

    hybrid_model.summary()

except Exception as e:
    print(f"Error building hybrid model: {str(e)}")


In [ ]:
# --- TRAINING HYBRID MODEL (WITHOUT VISUALIZATION) ---

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# --- 1. Callbacks ---
checkpoint = ModelCheckpoint(
    "best_hybrid_model.keras",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

early_stop = EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

callbacks = [checkpoint, early_stop, reduce_lr]

# --- 2. Train the model ---
history = hybrid_model.fit(
    train_generator,                     # training data generator
    validation_data=validation_generator, # validation data generator
    epochs=20,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator),
    callbacks=callbacks,
    verbose=1
)


🩺 Breast Cancer Classification – Hybrid Model Results
Model: EfficientNetV2B0 + Vision Transformer (ViT)
Input Size: 224 × 224 × 3
Epochs: 20
Optimizer: Adam
Loss Function: Binary Crossentropy
✅ Performance Metrics (Best Epoch)
Validation Accuracy: 92.41%
Validation Loss: 0.2111
Precision: 92.51%
Recall: 96.77%
📈 Training Observations
Accuracy steadily improved across epochs with learning rate adjustments.
Early stopping prevented overfitting after epoch 16.
Both precision and recall indicate strong performance for malignant detection.
📌 Key Takeaways
Hybrid model combines local feature extraction (EfficientNetV2) and global context (ViT).
Outperforms single-branch models in balanced recall and precision.
Further gains possible with fine-tuning both branches or advanced augmentation.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# History object from model.fit
history_obj = history
epochs = range(1, len(history_obj.history['accuracy']) + 1)

# Compute F1-score per epoch
train_precision = np.array(history_obj.history['precision'])
train_recall = np.array(history_obj.history['recall'])
val_precision = np.array(history_obj.history['val_precision'])
val_recall = np.array(history_obj.history['val_recall'])

train_f1 = 2 * (train_precision * train_recall) / (train_precision + train_recall + 1e-8)
val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall + 1e-8)

# --- 1. Accuracy ---
plt.figure(figsize=(8,5))
plt.plot(epochs, history_obj.history['accuracy'], 'b-', label='Train Accuracy')
plt.plot(epochs, history_obj.history['val_accuracy'], 'b--', label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# --- 2. Loss ---
plt.figure(figsize=(8,5))
plt.plot(epochs, history_obj.history['loss'], 'r-', label='Train Loss')
plt.plot(epochs, history_obj.history['val_loss'], 'r--', label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# --- 3. Precision ---
plt.figure(figsize=(8,5))
plt.plot(epochs, train_precision, 'g-', label='Train Precision')
plt.plot(epochs, val_precision, 'g--', label='Val Precision')
plt.title('Model Precision')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.legend()
plt.grid(True)
plt.show()

# --- 4. Recall ---
plt.figure(figsize=(8,5))
plt.plot(epochs, train_recall, 'm-', label='Train Recall')
plt.plot(epochs, val_recall, 'm--', label='Val Recall')
plt.title('Model Recall')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.legend()
plt.grid(True)
plt.show()

# --- 5. F1-Score ---
plt.figure(figsize=(8,5))
plt.plot(epochs, train_f1, 'c-', label='Train F1-Score')
plt.plot(epochs, val_f1, 'c--', label='Val F1-Score')
plt.title('Model F1-Score')
plt.xlabel('Epochs')
plt.ylabel('F1-Score')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# --- 1. Predict on validation set ---
val_preds = hybrid_model.predict(validation_generator, steps=len(validation_generator), verbose=1)
val_preds_classes = (val_preds > 0.5).astype(int).flatten()

# --- 2. Confusion Matrix ---
cm = confusion_matrix(validation_generator.classes, val_preds_classes)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(validation_generator.class_indices.keys()),
            yticklabels=list(validation_generator.class_indices.keys()))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# --- 3. Classification Report ---
report = classification_report(validation_generator.classes, val_preds_classes,
                               target_names=list(validation_generator.class_indices.keys()))
print("Classification Report:\n")
print(report)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Metrics from your classification report
classes = ['Benign', 'Malignant']
precision = [0.92, 0.93]
recall = [0.83, 0.97]
f1_score = [0.87, 0.95]

x = np.arange(len(classes))  # label locations
width = 0.25  # width of the bars

fig, ax = plt.subplots(figsize=(8,5))
rects1 = ax.bar(x - width, precision, width, label='Precision', color='skyblue')
rects2 = ax.bar(x, recall, width, label='Recall', color='lightgreen')
rects3 = ax.bar(x + width, f1_score, width, label='F1-Score', color='salmon')

# Add labels and title
ax.set_ylabel('Score')
ax.set_ylim(0,1)
ax.set_title('Classification Metrics per Class')
ax.set_xticks(x)
ax.set_xticklabels(classes)
ax.legend()

# Add value labels on top of bars
for rects in [rects1, rects2, rects3]:
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0,3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# --- 1. Predictions ---
val_preds = hybrid_model.predict(validation_generator, verbose=1)
val_preds_classes = (val_preds > 0.5).astype(int).flatten()
y_true = validation_generator.classes
class_names = list(validation_generator.class_indices.keys())

# --- 2. Compute metrics ---
accuracy = accuracy_score(y_true, val_preds_classes)
precision = precision_score(y_true, val_preds_classes)
recall = recall_score(y_true, val_preds_classes)
f1 = f1_score(y_true, val_preds_classes)

print("=== Classification Metrics ===")
print(f"Accuracy  : {accuracy:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")
print(f"F1-Score  : {f1:.4f}\n")

print("=== Detailed Classification Report ===")
print(classification_report(y_true, val_preds_classes, target_names=class_names))

# --- 3. Confusion Matrix ---
cm = confusion_matrix(y_true, val_preds_classes)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
import numpy as np

# --- Predictions ---
val_preds = hybrid_model.predict(validation_generator, verbose=1)
val_preds_classes = (val_preds > 0.5).astype(int).flatten()
y_true = validation_generator.classes
class_names = list(validation_generator.class_indices.keys())

# --- 1. Confusion Matrix ---
cm = confusion_matrix(y_true, val_preds_classes)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.savefig("confusion_matrix.png")
plt.close()  # close to avoid overlapping plots

# --- 2. Accuracy & Loss ---
plt.figure(figsize=(8,6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.title('Accuracy & Loss')
plt.legend()
plt.savefig("accuracy_loss.png")
plt.close()

# --- 3. Precision, Recall, F1 per class ---
precision, recall, f1, _ = precision_recall_fscore_support(y_true, val_preds_classes)
x = np.arange(len(class_names))
width = 0.25

plt.figure(figsize=(8,6))
plt.bar(x - width, precision, width, label='Precision')
plt.bar(x, recall, width, label='Recall')
plt.bar(x + width, f1, width, label='F1-score')
plt.xticks(x, class_names)
plt.ylabel('Score')
plt.title('Precision, Recall, F1 per Class')
plt.legend()
plt.savefig("precision_recall_f1.png")
plt.close()

print("All plots saved as images!")
